In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit

In [3]:
spark = SparkSession.builder.getOrCreate()

In [15]:
produtos = spark.read.csv('/content/drive/MyDrive/Material de apoio - M27/produtos.csv', header=True, inferSchema=True)
clientes = spark.read.csv('/content/drive/MyDrive/Material de apoio - M27/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('/content/drive/MyDrive/Material de apoio - M27/itens_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('/content/drive/MyDrive/Material de apoio - M27/pedidos.csv', header=True, inferSchema=True)

In [16]:
#join_pedidos_clientes_df = pedidos.join(clientes, pedidos['id_cliente'] == clientes['id_cliente'], 'inner')
#join_pedidos_clientes_df = pedidos.join(clientes, 'id_cliente', 'inner')

'''
outer
left_outer
right_outer
left
right
full
'''
#join_pedidos_clientes_df.show(5)
pedidos.show(5)
produtos.show(5)

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    delivered|2018-08-08 08:38:49|  2018-08-08 08:55:23|      2018-08-08 13:50:00| 2018-08-17 18:06:29|  2018-09

In [17]:
join_itens_pedidos_df = itens_pedido.join(pedidos, 'id_pedido').join(produtos, 'id_produto')
join_itens_pedidos_df.show(5)

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega| categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+--

In [20]:
join_pedidos_entregues_df = pedidos.filter(col('status_pedido') == 'canceled').join(clientes, 'id_cliente')
join_pedidos_entregues_df.show()

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+-----------+--------------------+--------------+
|          id_cliente|           id_pedido|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+-----------+--------------------+--------------+
|6d6b50b66d79f8082...|1b9ecfe83cdc25925...|     canceled|2018-08-04 14:29:27|  2018-08-07 04:10:26|                     NULL|                NULL|  2018-08-14 00:00:00|8ea097b1824dbd4d1...|       5514|           sao paulo|            SP|
|e3fe72696c4713d64...|714fb133a6730ab81...|     

In [22]:
pedidos_concat_df = pedidos.withColumn('id_pedido_cliente', concat(col('id_pedido'), lit('-'), col('id_cliente')))

colunas = pedidos_concat_df.columns
print(colunas)

colunas.remove('id_pedido_cliente')
colunas.insert(0, 'id_pedido_cliente')

pedidos_concat_df = pedidos_concat_df.select(colunas)
pedidos_concat_df.show(n=5, truncate=False)


['id_pedido', 'id_cliente', 'status_pedido', 'data_compra_pedido', 'data_aprovacao_pedido', 'data_envio_transportadora', 'data_entrega_cliente', 'data_estimada_entrega', 'id_pedido_cliente']
+-----------------------------------------------------------------+--------------------------------+--------------------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|id_pedido_cliente                                                |id_pedido                       |id_cliente                      |status_pedido|data_compra_pedido |data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+-----------------------------------------------------------------+--------------------------------+--------------------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b7cc49136f2d6af

In [23]:
join_itens_pedidos_df.write.mode('overwrite').option('header', 'true').parquet('/content/drive/MyDrive/Material de apoio - M27/join_itens_pedidos.parquet')

In [24]:
spark.read.option('header', 'true').parquet('/content/drive/MyDrive/Material de apoio - M27/join_itens_pedidos.parquet').show()

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+-------------------

In [25]:
spark.stop()